<a href="https://colab.research.google.com/github/Sahil-Chhabra-09/Book-Recommendation-System/blob/main/Books_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Books Recommendation System based on Cosine Similarity

In [60]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Book Recommendation System/BooksData.xlsx")

In [61]:
print(data.shape)
data.head()

(16559, 7)


,Unnamed: 0,Index,ID,BookTitle,Author,Genre,Summary
0,0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,3,4,1756,An Enquiry Concerning Human Understanding,David Hume,NaN,The argument of the Enquiry proceeds by a ser...
4,4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [62]:
data.isna().sum()

Unnamed: 0       0
Index            0
ID               0
BookTitle        0
Author        2382
Genre         3718
Summary          0
dtype: int64

In [63]:
data = data.drop(data[data['Genre'].isna()].index)
data = data.drop(data[data['Author'].isna()].index)

In [64]:
data.isna().sum()

Unnamed: 0    0
Index         0
ID            0
BookTitle     0
Author        0
Genre         0
Summary       0
dtype: int64

In [65]:
data.shape

(12055, 7)

In [66]:
print(data.Genre)
data.Genre[0]   #Genre was in the form of a json object

0        {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt":...
1        {"/m/06n90": "Science Fiction", "/m/0l67h": "N...
2        {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...
4        {"/m/03lrw": "Hard science fiction", "/m/06n90...
5        {"/m/098tmk": "War novel", "/m/016lj8": "Roman...
                               ...                        
16549                      {"/m/06n90": "Science Fiction"}
16551    {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction...
16555     {"/m/01jfsb": "Thriller", "/m/02xlf": "Fiction"}
16556                         {"/m/0xdf": "Autobiography"}
16558    {"/m/02ql9": "Epistolary novel", "/m/014dfn": ...
Name: Genre, Length: 12055, dtype: object


'{"/m/016lj8": "Roman \\u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children\'s literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}'

In [67]:
import json
genres_cleaned = []
for i in data['Genre']:
    genres_cleaned.append(list(json.loads(i).values()))   #converts json string to dictionary which is further converted into list
genres_cleaned[0:10]

[['Roman à clef',
  'Satire',
  "Children's literature",
  'Speculative fiction',
  'Fiction'],
 ['Science Fiction',
  'Novella',
  'Speculative fiction',
  'Utopian and dystopian fiction',
  'Satire',
  'Fiction'],
 ['Existentialism', 'Fiction', 'Absurdist fiction', 'Novel'],
 ['Hard science fiction',
  'Science Fiction',
  'Speculative fiction',
  'Fantasy',
  'Fiction'],
 ['War novel', 'Roman à clef'],
 ["Children's literature",
  'Fantasy',
  'Speculative fiction',
  'Bildungsroman',
  'Fiction'],
 ['Science Fiction', 'Speculative fiction'],
 ['Science Fiction', 'Speculative fiction'],
 ['Speculative fiction', 'Fiction', 'Novel'],
 ['Science Fiction',
  'Speculative fiction',
  "Children's literature",
  'Fiction']]

In [68]:
data['Genre'] = genres_cleaned

In [69]:
data.Summary[0]

' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself leader. N

In [70]:
import re   #regular expression matching
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)     #means any character that IS NOT a-z OR A-Z
    text = ' '.join(text.split())
    text = text.lower()
    return text

print(data['Summary'].apply(lambda x: clean_summary(x))[0])
data['Summary'] = data['Summary'].apply(lambda x: clean_summary(x))

old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy the animals revolt and drive the drunken and irresponsible mr jones from the farm renaming it animal farm they adopt seven commandments of animal ism the most important of which is all animals are equal snowball attempts to teach the animals reading and writing food is plentiful and the farm runs smoothly the pigs elevate themselves to positions of leadership and set aside special food items ostensibly for their personal health napoleon takes the pups from the farm dogs and trains them privately napoleon and snowball struggle for leadership when snowball announces his plans to build a windmill napoleon has his dogs chase snowball away and declares himself leader napoleon enacts changes to the gov

In [71]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
data['GenreString'] = data['Genre'].apply(lambda x: ' '.join(x))

In [73]:
data['GenreString'][0]

"Roman à clef Satire Children's literature Speculative fiction Fiction"

In [74]:
#get a combined text that includes author's name and associated genres
data["combined_text"] = data["Summary"] + " " + data["Author"] + " " + data["GenreString"]

In [75]:
data["combined_text"]

0        old major the old boar on the manor farm calls...
1        alex a teenager living in near future england ...
2        the text of the plague is divided into five pa...
4        the novel posits that space around the milky w...
5        the book tells the story of paul b umer a germ...
                               ...                        
16549    the story starts with former government agent ...
16551    the series follows the character of nick stone...
16555    the reader first meets rapp while he is doing ...
16556    the book follows very rough chronological orde...
16558    makar devushkin and varvara dobroselova are se...
Name: combined_text, Length: 12055, dtype: object

In [76]:
# removing stopwords
stopwords_english = stopwords.words("english")
data['text_without_stopwords'] = data['combined_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_english)]))
data['text_without_stopwords'][0]

"old major old boar manor farm calls animals farm meeting compares humans parasites teaches animals revolutionary song beasts england major dies two young pigs snowball napoleon assume command turn dream philosophy animals revolt drive drunken irresponsible mr jones farm renaming animal farm adopt seven commandments animal ism important animals equal snowball attempts teach animals reading writing food plentiful farm runs smoothly pigs elevate positions leadership set aside special food items ostensibly personal health napoleon takes pups farm dogs trains privately napoleon snowball struggle leadership snowball announces plans build windmill napoleon dogs chase snowball away declares leader napoleon enacts changes governance structure farm replacing meetings committee pigs run farm using young pig named squealer mouthpiece napoleon claims credit windmill idea animals work harder promise easier lives windmill violent storm animals find windmill annihilated napoleon squealer convince ani

In [77]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()
count_matrix = cv.fit_transform(data['text_without_stopwords'])

In [78]:
count_matrix.shape

(12055, 112000)

In [79]:
from sklearn.metrics.pairwise import cosine_similarity
cosine = cosine_similarity(count_matrix, count_matrix)

In [80]:
# Term frequency inverse document matrix

tf = TfidfVectorizer(analyzer = "word", ngram_range=(1,2), min_df=0, stop_words='english')

tfidf_matrix = tf.fit_transform(data['text_without_stopwords'])

cosine2 =  cosine_similarity(tfidf_matrix, tfidf_matrix)

In [88]:
def get_title_from_index(Index):
    return data[data.Index == Index].BookTitle.values[0]
def get_index_from_title(BookTitle):
    return data[data.BookTitle == BookTitle]["Index"].values[0]

def get_recommendations(book):
    book_index = get_index_from_title(book)
    similar_books = list(enumerate(cosine[book_index]))
    # print(similar_books)
    sortedbooks = sorted(similar_books, key = lambda x:x[1], reverse=True)[1:]
    # print(sortedbooks)
    i = 0
    for book in sortedbooks:
        if data[data.Index==book[0]].shape[0]==0:
          continue;
        title = data[data.Index==book[0]].BookTitle
        author = data[data.Index==book[0]].Author
        print(title + " by " + author)
        i = i+1
        if i>10:
            break

In [89]:
def get_title_from_index(Index):
    return data[data.Index == Index].BookTitle.values[0]
def get_index_from_title(BookTitle):
    return data[data.BookTitle == BookTitle]["Index"].values[0]

def get_recommendations2(book):
    book_index = get_index_from_title(book)
    similar_books = list(enumerate(cosine2[book_index]))
    # print(similar_books)
    sortedbooks = sorted(similar_books, key = lambda x:x[1], reverse=True)[1:]
    # print(sortedbooks)
    i = 0
    for book in sortedbooks:
        if data[data.Index==book[0]].shape[0]==0:
          continue;
        title = data[data.Index==book[0]].BookTitle
        author = data[data.Index==book[0]].Author
        print(title + " by " + author)
        i = i+1
        if i>10:
            break

In [90]:
print(get_recommendations('Dune'))

4640    Planet of Judgment by Joe Haldeman
dtype: object
4651    Sassinak by Anne McCaffrey
dtype: object
11272    My Booky Wook by Russell Brand
dtype: object
2354    The Gold Bug Variations by Richard Powers
dtype: object
6655    Last Day in Limbo by Peter O'Donnell
dtype: object
1937    Idoru by William Gibson
dtype: object
7937    Star-Begotten by H. G. Wells
dtype: object
2179    Snow Country by Yasunari Kawabata
dtype: object
6541    Tragedy Day by Gareth Roberts
dtype: object
4118    Threshold by David R. Palmer
dtype: object
3899    Such a Long Journey by Rohinton Mistry
dtype: object
None


In [91]:
print(get_recommendations2('Dune'))

4651    Sassinak by Anne McCaffrey
dtype: object
10125    Sticky Wicket at Blandings by P. G. Wodehouse
dtype: object
3985    The Swarm War by Troy Denning
dtype: object
2790    The Edge of the Cloud by K. M. Peyton
dtype: object
1978    Up the Down Staircase by Bel Kaufman
dtype: object
6399    Ancestors of Avalon by Marion Zimmer Bradley
dtype: object
10039    Evening Class by Maeve Binchy
dtype: object
1141    Galactic Pot-Healer by Philip K. Dick
dtype: object
466    Dreamcatcher by Stephen King
dtype: object
10813    Master of the Moon by Angela Knight
dtype: object
413    The Newcomes by William Makepeace Thackeray
dtype: object
None


In [92]:
print(get_recommendations2('A Fire Upon the Deep'))

667    Trainspotting by Irvine Welsh
dtype: object
727    Trumps of Doom by Roger Zelazny
dtype: object
254    The Body by Stephen King
dtype: object
3123    The Haunting of Hill House by Shirley Jackson
dtype: object
426    War and Peace by Leo Tolstoy
dtype: object
1347    Congo by Michael Crichton
dtype: object
10814    Dark Melody by Christine Feehan
dtype: object
6884    The Assassin King by Elizabeth Haydon
dtype: object
7800    Ishmael by Barbara Hambly
dtype: object
5892    The Harsh Cry of the Heron by Gillian Rubinstein
dtype: object
684    The Turn of the Screw by Henry James
dtype: object
None
